# snm3C Mapping Summary

This notebook provides a quick overview of some key mapping metrics. You can customize it by yourself.

[**See documentation about mapping metrics here.**](https://app.gitbook.com/@hq-1/s/mc/mapping-summary-and-metrics)

## Parameters

## Prepare

In [ ]:
output_dir = ''
plate_col = 'Plate'
color_quantile = (0.05, 0.95)

### Import

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from cemba_data.mapping import cutoff_vs_cell_remain, plot_on_plate
from cemba_data.utilities import get_configuration


def distplot_and_plate_view(data, hue, color_quantile=color_quantile):
    fig1, (vmin, vmax) = cutoff_vs_cell_remain(data=data[hue].dropna(), 
                                               bins=50, kde=False,
                                               xlim_quantile=color_quantile)

    fig2, plate_names, plate_datas = plot_on_plate(
        data=data,
        hue=hue,
        groupby=plate_col,
        vmin=vmin,
        vmax=vmax,
        aggregation_func=lambda i: i.mean())
    return

In [ ]:
# plot defaults
sns.set_context(context='notebook')
plt.rc('figure', dpi=200)

### Load

In [ ]:
output_dir = pathlib.Path(output_dir)
mapping_summary = pd.read_csv(output_dir / 'stats/MappingSummary.csv.gz', index_col=0)
config = get_configuration(output_dir / 'mapping_config.ini')

In [ ]:
mapping_summary.columns

## Summary

In [ ]:
# conventional basic check, change as you need
mccc_cutoff = 0.03
high_mccc = mapping_summary['mCCCFrac'] > mccc_cutoff

miseq_guess = mapping_summary['FinalmCReads'].mean() < 50000
reads_cutoff = 100 if miseq_guess else 500000
low_reads = mapping_summary['FinalmCReads'] < reads_cutoff

success = ~high_mccc & ~low_reads
n_cell = mapping_summary.shape[0]
n_plate = mapping_summary['Plate'].unique().size
total_wells = n_plate * 384

In [ ]:
print(f"""
This library seems to be a {'MiSeq' if miseq_guess else 'NovaSeq'} library.

Cells
    {n_plate}\t plates
    {total_wells}\t wells (total cell number in theory)

    {n_cell} ({n_cell / total_wells * 100:.1f}%)\t cells having mapping metric
    {success.sum()} ({success.sum() / total_wells * 100:.1f}%)\t cells passed basic QC (mCCC and # of final reads)
    {high_mccc.sum()} ({high_mccc.sum() / total_wells * 100:.1f}%)\tcells having high mCCC frac (> {mccc_cutoff})
    {low_reads.sum()} ({low_reads.sum() / total_wells * 100:.1f}%)\tcells having low number of final mC reads (< {reads_cutoff}).

Reads
    {mapping_summary['CellInputReadPairs'].sum()*2:.0f}\tTotal Input Reads (R1 & R2)
    {mapping_summary['CellInputReadPairs'].mean()*2:.0f}\tAverage Input Reads for cells having metric (R1 & R2)
    {mapping_summary['FinalmCReads'].sum():.0f}\tTotal Final Reads (R1 & R2)
    {mapping_summary['FinalmCReads'].mean():.0f}\tAverage Final Reads for cells having metric (R1 & R2)

    {mapping_summary['R1MappingRate'].mean():.1f}%\tAverage R1 Mapping Rate for cells having metric
    {mapping_summary['R2MappingRate'].mean():.1f}%\tAverage R2 Mapping Rate for cells having metric
    {mapping_summary['R1DuplicationRate'].mean():.1f}%\tAverage R1 PCR Duplicate Rate for cells having metric
    {mapping_summary['R2DuplicationRate'].mean():.1f}%\tAverage R2 PCR Duplicate Rate for cells having metric

Contacts
    {mapping_summary['CisLongRatio'].mean(): .2f}\tAverage Cis Long Contacts ratio for cells having metric
    {mapping_summary['TransRatio'].mean(): .2f}\tAverage Trans Contacts ratio for cells having metric
""")

## mC Fraction

### mCCC

In [ ]:
distplot_and_plate_view(mapping_summary, hue='mCCCFrac')

### mCH

In [ ]:
distplot_and_plate_view(mapping_summary, hue='mCHFrac')

### mCG

In [ ]:
distplot_and_plate_view(mapping_summary, hue='mCGFrac')

## FASTQ Metric

### CellInputReadPairs

In [ ]:
distplot_and_plate_view(mapping_summary, hue='CellInputReadPairs')

## Cell Barcode Portion

In [ ]:
distplot_and_plate_view(mapping_summary, hue='CellBarcodeRatio')

## Mapping Rate

### R1 Mapping Rate

In [ ]:
distplot_and_plate_view(mapping_summary, hue='R1MappingRate')

### R2 Mapping Rate

In [ ]:
distplot_and_plate_view(mapping_summary, hue='R2MappingRate')

## PCR Duplication Rate

### R1 PCR Duplication Rate

In [ ]:
distplot_and_plate_view(mapping_summary, hue='R1DuplicationRate')

### R2 PCR Duplication Rate

In [ ]:
distplot_and_plate_view(mapping_summary, hue='R2DuplicationRate')

## Final mC Reads

In [ ]:
distplot_and_plate_view(mapping_summary, hue='FinalmCReads')

## Chromatin Contacts

### Cis-long Counts

In [ ]:
distplot_and_plate_view(mapping_summary, hue='CisLongContact')

### Trans Counts

In [ ]:
distplot_and_plate_view(mapping_summary, hue='TransContact')

## Chromatin Contacts Ratio

### Cis-long Ratio

In [ ]:
distplot_and_plate_view(mapping_summary, hue='CisLongRatio')

### Trans Ratio

In [ ]:
distplot_and_plate_view(mapping_summary, hue='TransRatio')

## Mapping config

In [ ]:
config